In [1]:
# coding: utf-8
# наше всё
import numpy as np
import pandas as pd

#настройки pandas, с которыми лучше почти всегда
pd.set_option('display.max_rows', 45000)
pd.set_option('display.max_columns', 50000)
pd.set_option('display.max_colwidth', 5000)

# будем отображать графики прямо в jupyter'e
import seaborn as sns
import matplotlib.pyplot as plt

import sys
sys.path.append('//')

import data_load as dl

#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 12, 9

In [2]:
# словарь конвертирует исход в эло-результат
score_dct={
    'Win': 1,
    'Lose': 0,
    'Draw': 0.5,
    'Unknown': 0
        }


def get_season(lst):
    if len(lst)==5:
        return lst[3]
    else:
        return lst[2]
def get_division(txt):
    l=txt.split('.')
    return l[1]
def get_competition(txt):
    l=txt.split('.')
    return l[0]

def formatting(df):
    # дата в нормальном формате
    df['date']=df['date'].astype('datetime64[ns]')
    
    # если результат игры NULL, ставим -1
    df['home_result']=np.where(
        (df['gh']>df['ga']), 'Win', 
                                    np.where(
                                        (df['gh']<df['ga']), 'Lose', 'Draw'
                                            )
                                )
    df['away_result']=np.where(
        (df['gh']>df['ga']), 'Lose', 
                                    np.where(
                                        (df['gh']<df['ga']), 'Win', 'Draw'
                                            )
                                )
    
    df=df.rename(columns={
                        'full_time': 'result_kind',
                        'gh': 'home_score',
                        'ga': 'away_score',
                        'home': 'home_team',
                        'away': 'away_team'
                          })

    
    return df

def team_table(df):
    # делаем датасет, где каждая в фокусе команда, а не результат матча
    # сначала для хозяев, потом гостей
    df_home=df.copy()

    df_home['team'] = df_home['home_team']
    df_home['opponent'] = df_home['away_team']
    df_home['result'] = df_home['home_result']
    df_home['team_score'] = df_home['home_score']
    df_home['opponent_score'] = df_home['away_score']
    df_home['venue'] = 'Home'
    
    df_home['full_team_name'] = df_home['home_ident']
    df_home['opponent_full_team_name'] = df_home['away_ident']
    df_home['country'] = df_home['home_country']
    df_home['opponent_country'] = df_home['away_country']
    df_home['country_code'] = df_home['home_code']
    df_home['opponent_country_code'] = df_home['away_code']

    df_home=df_home[[
        'match_id', 'date', 'competition', 'level',
        'team', 'opponent', 'venue', 
        'result', 'result_kind',
        'team_score', 'opponent_score',
        'continent', 'country', 'opponent_country',
        'country_code', 'opponent_country_code',
        'full_team_name', 'opponent_full_team_name',
        'file'
                    ]]
    
    df_away=df.copy()
    
    df_away['team'] = df_away['away_team']
    df_away['opponent'] = df_away['home_team']
    df_away['result'] = df_away['away_result']
    df_away['team_score'] = df_away['away_score']
    df_away['opponent_score'] = df_away['home_score']
    df_away['venue'] = 'Away'

    df_away['full_team_name'] = df_away['away_ident']
    df_away['opponent_full_team_name'] = df_away['home_ident']
    df_away['country'] = df_away['away_country']
    df_away['opponent_country'] = df_away['home_country']
    df_away['country_code'] = df_away['away_code']
    df_away['opponent_country_code'] = df_away['home_code']
    
    df_away=df_away[[
    'match_id', 'date', 'competition', 'level',
    'team', 'opponent', 'venue', 
    'result', 'result_kind',
    'team_score', 'opponent_score',
    'continent', 'country', 'opponent_country',
    'country_code', 'opponent_country_code',
    'full_team_name', 'opponent_full_team_name',
    'file'
                ]]

    team_df=pd.concat([df_home, df_away]).sort_values(by='date')
    l=team_df.sort_values(['date'], ascending=[True]) \
                 .groupby(['team']) \
                 .cumcount() + 1
    team_df['game_number'] = l.values

    return team_df

# retunr classic Elo propabilities
# elo_prob(2882, 2722) -> 0.7152 (72% chanses Carlsen (2882) to beat Wan Hao (2722))
def elo_prob(rw, rb):
    try:
        rw=float(rw)
        rb=float(rb)
        res=1/(1+np.power(10, (rb-rw)/400))
    except:
        0.5
    return res

# изменение рейтинга по фактическому выступлению
def elo_rating_changes(rating, opponent_rating, score, games):
    # чтобы новички быстрее набирали рейтиг
    if games<=30:
        K=40
    else:
        # у топ-топов всё мееяется медленней
        if rating>2500:
            K=10
        elif rating<=2500:
            K=20
            
    expectation=elo_prob(rating, opponent_rating)
    new_rating=rating+K*(score-expectation)
    
    return new_rating

# на вход получаем список сущностей , 
# на выходе - стринга сочетаний сущеностей 
def get_all_sets(ll):
    rr=[]
    r=[]
    for i in ll:
        for j in ll:
            if i!=j:
                rr.append(str(i)+'-'+str(j))
    return rr


def count_elo_rating(team_df):
    # создаём словарь с текущими рейтингами, будем обновлять его после каждого матча
    teams=team_df['team'].value_counts().index
    
    # начальный рейтинг пусть будет 2400
    default_ratings=np.ones(len(teams))*2400
    current_rating_dict=dict(zip(teams, default_ratings))
    
    # добавдляем словарь статистики персональных встреч
    current_stat_dict=dict({'team1-team2': np.zeros(5)})

    # результаты для добавления в датафрейм
    ratings_lst=[]
    opponent_ratings_lst=[]
    ratings_chng_lst=[]
    wins_lst=[]
    draws_lst=[]
    loses_lst=[]
    scored_lst=[]
    mised_lst=[]
    
    # если сортировка будет не хронологической, то упс
    team_df=team_df.sort_values(by=['date', 'team'])
    
    # цикл по всем матчам
    for match in team_df.values:
        # если данных нет, не нужно менять рейтинг никак
#         пример строки 
#         699 Timestamp('2000-04-15 00:00:00') 'malaysia' 'national' 'Perlis'
#          'Selangor FC' 'Away' 'Draw' 'F' 2 2 'Asia' 'malaysia' 'malaysia' 'MY'
#          'MY' 'Perlis (Malaysia)' 'Selangor FC (Malaysia)'
#          'data/football-data/data/results/malaysia.csv' 1
        wins=0
        draws=0
        loses=0
        scored=0
        mised=0
        if match[7]!='Unknown':
            curr_t_str=str(match[4])+'-'+str(match[5])
            
            if (curr_t_str not in current_stat_dict.keys()):
                current_stat_dict.update({
                                        curr_t_str:np.zeros(5)
                                            })

            if match[7]=='Win':
                current_stat_dict[curr_t_str][0]=current_stat_dict[curr_t_str][0]+1
            elif match[7]=='Draw':
                current_stat_dict[curr_t_str][1]=current_stat_dict[curr_t_str][1]+1
                
            elif match[7]=='Lose':
                current_stat_dict[curr_t_str][2]=current_stat_dict[curr_t_str][2]+1
                
                
            wins=current_stat_dict[curr_t_str][0]
            wins_lst.append(wins) 
            draws=current_stat_dict[curr_t_str][1]
            draws_lst.append(draws)
            loses=current_stat_dict[curr_t_str][2]
            loses_lst.append(loses)
            
            
            current_stat_dict[curr_t_str][3]=current_stat_dict[curr_t_str][3]+match[9]
            scored=current_stat_dict[curr_t_str][3]
            scored_lst.append(scored)
            
            current_stat_dict[curr_t_str][4]=current_stat_dict[curr_t_str][4]+match[10]
            mised=current_stat_dict[curr_t_str][4]
            mised_lst.append(mised)
            
            # ожидаемый исход матча
            expectation=elo_prob(current_rating_dict[match[4]], 
                                 current_rating_dict[match[5]]
                                )
            # пересчёт нового рейтинга
            new_rating=elo_rating_changes(
                                    current_rating_dict[match[4]], 
                                    current_rating_dict[match[5]],
                                    score_dct[match[7]], match[19]
                                           )

            # изменение рейтинга    
            changing=new_rating-current_rating_dict[match[4]]

            # обновление рабочего словаря
            current_rating_dict.update({match[4]: new_rating})

            # а теперь пересчитаем рейтинг оппонента
            opponent_rating=current_rating_dict[match[5]]-changing
            
        else:
            new_rating=current_rating_dict[match[4]]
            changing=0
            opponent_rating=current_rating_dict[match[5]]
            
            wins=current_stat_dict[curr_t_str][0]
            draws=current_stat_dict[curr_t_str][1]
            loses=current_stat_dict[curr_t_str][2]
            scored=current_stat_dict[curr_t_str][3]
            mised=current_stat_dict[curr_t_str][4]
            
            wins_lst.append(wins)
            draws_lst.append(draws)
            loses_lst.append(loses)
            scored_lst.append(scored)
            mised_lst.append(mised)


        # списки рейтингов для передачи в датафрейм
        ratings_lst.append(new_rating)
        opponent_ratings_lst.append(opponent_rating)
        ratings_chng_lst.append(changing)
        
    # рейтинги ПО итогу матча
    team_df['rating'] = ratings_lst
    team_df['opponent_rating'] = opponent_ratings_lst
    team_df['rating_changing'] = ratings_chng_lst   
    # история личных встреч ПО итогу матча
    team_df['wins'] = wins_lst
    team_df['draws'] = draws_lst
    team_df['loses'] = loses_lst
    team_df['scored'] = scored_lst
    team_df['mised'] = mised_lst
    
    return team_df, current_rating_dict, current_stat_dict

def elo_predict(team, opponent, draw_share):
    try:
        rating=current_rating_dict[team]
        opponent_rating=current_rating_dict[opponent]
        prob=elo_prob(rating, opponent_rating)
        # пока прибьём гвоздями
        # как с данными будет лучше, для каждой лиги\сезона свой будет
#         draw_share=0.26768295275757964
#         draw_share=0.1
#         draw_share=0.05
        draw_balance=0.5

        if prob>draw_balance+draw_share/2:
            return 'Win'
        elif prob<draw_balance-draw_share/2:
            return 'Lose'
        else:
            return 'Draw'
    except:
        return 'Unknown'
    
def simple_predict(team, opponent):
    return 'Win'

def random_predict(team, opponent):
    return np.random.choice(['Win', 'Lose', 'Draw'])

# insert data in sql table
def insert_df(df, table_name):
    import datetime 
    now=datetime.datetime.now() 
    df['inserted_at']=now
    
    postgresql_engine=get_engine()
    
    df.to_sql(table_name, con=postgresql_engine, if_exists='append')   
    print('Ready: ', len(df), ' rows inserted')   

In [7]:
%%time
# формируем датасет
# все доступные файлы
root='data/football-data/data/results/'
import os
file_lst=[]
for path, subdirs, files in os.walk(root):
    for name in files:
        if ('.csv' in name)==True:
            file_lst.append(os.path.join(path, name))
            
# всё собираем в один даатфрейм
df_tot=pd.DataFrame()
for file in file_lst[-3:]:
#     print(file)
    b=pd.read_csv(file)
    b['file']=file
    df_tot=pd.concat([df_tot, b])
df_tot['match_id']=range(len(df_tot))
df_tot['match_id']='fdr_'+df_tot['match_id'].astype('str')

CPU times: user 30.7 ms, sys: 306 µs, total: 31 ms
Wall time: 60.7 ms


In [8]:
len(df_tot)

2910

In [20]:
# собираем пак для записи в SQL
df_tot['source'] = 'file:_'+df_tot['file']
columns_to_sql = [
       'match_id',
       'home', 'away', 'date', 'gh', 'ga', 'full_time', 
       'level', 'competition',
       'source'
            ]

df_to_sql = df_tot[columns_to_sql]

In [21]:
df_to_sql.sample(3)

,match_id,home,away,date,gh,ga,full_time,level,competition,source
978,fdr_978,Coleraine FC,1 FC Lok Leipzig,1977-09-14,1,4,F,international,UEFA CW,file:_data/football-data/data/results/uefa-cw.csv
1594,fdr_1594,Avenir Beggen,Hamburger SV,1987-09-15,0,5,F,international,UEFA CW,file:_data/football-data/data/results/uefa-cw.csv
1699,fdr_1699,Lech Poznan,FC Barcelona,1988-11-09,4,5,F,international,UEFA CW,file:_data/football-data/data/results/uefa-cw.csv


In [22]:
len(df_to_sql)

2910

In [24]:
main_engine=dl.get_engine('config.yaml')

In [25]:
df_to_sql[0:3].to_sql('footbal_data_temp1', con=main_engine, if_exists='append', index=False)

In [26]:
dl.get_data("""

SELECT *
FROM footbal_data_temp1
limit 5

""")

,match_id,home,away,date,gh,ga,full_time,level,competition,source
0,fdr_0,1 FC Frankfurt Oder,Ruda Hvzda Brno,1960-07-31,2,1,F,international,UEFA CW,file:_data/football-data/data/results/uefa-cw.csv
1,fdr_1,Ruda Hvzda Brno,1 FC Frankfurt Oder,1960-08-11,2,0,F,international,UEFA CW,file:_data/football-data/data/results/uefa-cw.csv
2,fdr_2,Rangers FC,Ferencvarosi Tc,1960-09-28,4,2,F,international,UEFA CW,file:_data/football-data/data/results/uefa-cw.csv


In [8]:
dl.get_data("""

SELECT count(*)
FROM footbal_data
limit 5

""")

,count
0,1078214


In [10]:
dl.get_data("""

SELECT distinct table_schema FROM information_schema.columns

limit 5

""")

,table_schema
0,public
1,pg_catalog
2,information_schema


In [14]:
dl.get_data("""

drop table main_data


""")

""
